In [1]:
import swat
import pandas as pd
import numpy as np


fetch_opts = dict(to=1000000, maxrows=1000000)
dpp_opts = dict(distinctCountLimit=100000, misraGries=False)

In [2]:
server = "rdcgrd133.unx.sas.com"
port = 16271
s = swat.CAS(server, port)

In [3]:
s.loadactionset(actionset='dataPreprocess')
s.loadactionset(actionset='dataSciencePilot')
s.loadactionset(actionset='astore')
s.loadactionset(actionset='dataStep')

NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataPreprocess'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.052957 seconds
NOTE:       cpu time                0.152977 seconds (288.87%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  7.46M (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'dataSciencePilot'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time               0.052375 seconds
NOTE:       cpu time                0.028994 seconds (55.36%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  1.48M (0.00%)
NOTE: Executing action 'builtins.loadActionSet'.
NOTE: Added action set 'astore'.
NOTE: Action 'builtins.loadActionSet' used (Total process time):
NOTE:       real time     

[actionset]

 'dataStep'

+ Elapsed: 0.00422s, user: 0.004s, sys: 0.005s, mem: 1.44mb

In [4]:
input_tbl = "kdd98"
s.upload("U:\\data\\kdd98.csv", casout=dict(name=input_tbl, replace=True))

NOTE: Executing action 'table.loadTable'.
NOTE: Cloud Analytic Services made the uploaded file available as table KDD98 in caslib CASUSERHDFS(bigebr).
NOTE: Action 'table.loadTable' used (Total process time):
NOTE:       real time               0.809864 seconds
NOTE:       cpu time                9.863503 seconds (1217.92%)
NOTE:       data movement time      0.348497 seconds
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  698.17M (0.04%)
NOTE:       bytes moved             414.13M
NOTE: The table KDD98 has been created in caslib CASUSERHDFS(bigebr) from binary data uploaded to Cloud Analytic Services.
NOTE: Action 'table.upload' used (Total process time):
NOTE:       real time               4.439232 seconds
NOTE:       cpu time                10.173454 seconds (229.17%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  698.17M (0

[caslib]

 'CASUSERHDFS(bigebr)'

[tableName]

 'KDD98'

[casTable]

 CASTable('KDD98', caslib='CASUSERHDFS(bigebr)')

+ Elapsed: 4.44s, user: 8s, sys: 2.17s, mem: 698mb

In [5]:
s.tableInfo()

NOTE: Executing action 'table.tableInfo'.
NOTE: Action 'table.tableInfo' used (Total process time):
NOTE:       real time               0.004767 seconds
NOTE:       cpu time                0.013999 seconds (293.66%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  2.40M (0.00%)


,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Repeated,View,MultiPart,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,KDD98,95412,481,0,utf-8,2020-10-12T16:26:51-04:00,2020-10-12T16:26:51-04:00,2020-10-12T16:26:51-04:00,UTF8,1.918154e+09,...,0,0,0,,,0,bigebr,,2020-10-12T16:26:51-04:00,1.918154e+09


In [6]:
target = 'targetb'
event = "1"

# 1. Identify groups of variables using a comprehensive set of statistical metrics

In [21]:
s.dataSciencePilot.exploreData(table=input_tbl, target=target, casout=dict(name='avapt_out', replace=True));
avapt_df = s.fetch(table='avapt_out', **fetch_opts)['Fetch']

low_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 1')['Variable'])
low_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 2')['Variable'])
low_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 1 & MissingRated == 3')['Variable'])
medium_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 1')['Variable'])
medium_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 2')['Variable'])
medium_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 2 & MissingRated == 3')['Variable'])
high_card_low_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 1')['Variable'])
high_card_medium_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 2')['Variable'])
high_card_high_miss_nominal_vars = list(avapt_df.query('CardinalityRated == 3 & MissingRated == 3')['Variable'])


low_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 1')['Variable'])
low_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 2')['Variable'])
low_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 1 & MissingRated == 3')['Variable'])
medium_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 1')['Variable'])
medium_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 2')['Variable'])
medium_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 2 & MissingRated == 3')['Variable'])
high_skew_low_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 1')['Variable'])
high_skew_medium_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 2')['Variable'])
high_skew_high_miss_interval_vars = list(avapt_df.query('SkewnessRated == 3 & MissingRated == 3')['Variable'])



NOTE: Executing action 'dataSciencePilot.exploreData'.
NOTE: Action 'dataSciencePilot.exploreData' used (Total process time):
NOTE:       real time               1.785026 seconds
NOTE:       cpu time                23.394442 seconds (1310.59%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  328.29M (0.02%)
NOTE:       bytes moved             175.51K
NOTE: Executing action 'table.fetch'.
NOTE: Action 'table.fetch' used (Total process time):
NOTE:       real time               0.019673 seconds
NOTE:       cpu time                0.037995 seconds (193.13%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  7.72M (0.00%)


In [36]:
# example non-interaction pipelines 

pipelines = []
if len(high_card_low_miss_nominal_vars) > 0:
    p = dict(name='hclm',
              inputs = high_card_low_miss_nominal_vars,
              targets = target,
              events = event,
              impute = dict(method = 'mode'),
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=False,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p1)

if len(high_card_medium_miss_nominal_vars) > 0:
    p = dict(name='hcmm',
              inputs = high_card_medium_miss_nominal_vars,
              targets = target,
              events = event,
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=True,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p)
    
if len(high_card_high_miss_nominal_vars) > 0:
    p = dict(name='hchm',
              inputs = high_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              mapInterval = dict(method='empBayes',
                                 args = dict(includeMissingLevel=True,
                                             rareThreshold=5)),
              discretize = dict(method = 'bucket',
                                args = dict(nBins = 5)))
    pipelines.append(p)

if len(low_card_high_miss_nominal_vars) > 0:
    p = dict(name='lchm',
              inputs = low_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              cattrans = dict(method = 'label', 
                              args = dict(overrides=dict(binMissing=True))))
    pipelines.append(p)

if len(low_card_low_miss_nominal_vars) > 0:
    p = dict(name='lclm',
              inputs = low_card_high_miss_nominal_vars,
              targets = target,
              events = event,
              impute = dict(method = 'mode'),
              cattrans = dict(method = 'label', 
                              args = dict(overrides=dict(binMissing=True))))
    pipelines.append(p)
    
if len(high_skew_high_miss_interval_vars) > 0:
    p = dict(name='hshm',
              inputs = high_skew_high_miss_interval_vars,
              targets = target,
              events = event,
              discretize = dict(method = 'mdlp', args = dict(nBins = 5,
                                                             overrides=dict(binMissing=True))))
    pipelines.append(p)


In [37]:
# example interaction pipelines 

# e.g. among low skew, low-miss vs low card, low-miss
if len(low_skew_low_miss_interval_vars) > 0 and len(low_card_low_miss_nominal_vars):
    output_opts = dict(output=dict(noScoreTable=True, noScoreCode=True, noTable=True))
    p1 = dict(name='tr1', inputs=low_skew_low_miss_interval_vars,
              discretize=dict(method='quantile', args=dict(nBins=5)),
              **output_opts)
    p2 = dict(name='tr1', inputs=low_card_low_miss_nominal_vars,
              cattrans = dict(method='label'),
              **output_opts)
    tar_p = dict(name='tar_p', inputs=target, cattrans=dict(method='label'),
                 **output_opts)
    p = dict(name='inter1',
             interaction = dict(
                 method = 'cross',
                 synthesizer = 'nominal',
                 inputs = [p1['name'], p2['name']],
                 targets = tar_p['name']
             ),
             cattrans = dict(method='dtree', args=dict(nBins=5)))
    pipelines.append(p1)
    pipelines.append(p2)
    pipelines.append(tar_p)
    pipelines.append(p)
              
        

In [38]:
pipelines

[{'name': 'tr1',
  'inputs': ['HV1', 'HV2', 'IC1', 'IC2', 'IC3', 'IC4', 'IC5', 'CONTROLN'],
  'discretize': {'method': 'quantile', 'args': {'nBins': 5}},
  'output': {'noScoreTable': True, 'noScoreCode': True, 'noTable': True}},
 {'name': 'hcmm',
  'inputs': ['RFA_7',
   'RFA_9',
   'RFA_11',
   'RFA_12',
   'RFA_14',
   'RFA_16',
   'RFA_18',
   'AGE',
   'NEXTDATE',
   'TIMELAG'],
  'targets': 'targetb',
  'events': '1',
  'mapInterval': {'method': 'empBayes',
   'args': {'includeMissingLevel': True, 'rareThreshold': 5}},
  'discretize': {'method': 'bucket', 'args': {'nBins': 5}}},
 {'name': 'hchm',
  'inputs': ['RFA_10',
   'RFA_13',
   'RFA_17',
   'RFA_19',
   'RFA_20',
   'RFA_21',
   'RFA_22',
   'RFA_23',
   'RFA_24'],
  'targets': 'targetb',
  'events': '1',
  'mapInterval': {'method': 'empBayes',
   'args': {'includeMissingLevel': True, 'rareThreshold': 5}},
  'discretize': {'method': 'bucket', 'args': {'nBins': 5}}},
 {'name': 'lchm',
  'inputs': ['MAILCODE',
   'PVASTATE',


In [39]:
s.transform(table=input_tbl,
            pipelines = pipelines,
            saveState = dict(name='scoring_model', replace=True),
            **dpp_opts)

NOTE: Executing action 'dataPreprocess.transform'.
NOTE: 194620 bytes were written to the table "scoring_model" in the caslib "CASUSERHDFS(bigebr)".
NOTE: Action 'dataPreprocess.transform' used (Total process time):
NOTE:       real time               1.673252 seconds
NOTE:       cpu time                29.897456 seconds (1786.79%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  1.68G (0.10%)
NOTE:       bytes moved             190.77K


[TransInfo]

 Transformation Requests for KDD98
 
   GivenName ActualName Interaction InputTrans1 InputTrans2 TargetTrans  \
 0      hcmm                     no                                       
 1      hchm                     no                                       
 2      lchm                     no                                       
 3      lclm                     no                                       
 4      hshm                     no                                       
 5    inter1                    yes         tr1         tr1       tar_p   
 
    NTransVars  NEvalVars PhaseOrder ImputeMethod      MapInterval  \
 0          10          1    Map-Dis               Empirical Bayes   
 1           9          1    Map-Dis               Empirical Bayes   
 2          90          1        Cat                                 
 3          90          1    Imp-Cat         Mode                    
 4           8          1        Dis                                 
 5          64          1        Cat                                 
 
             DisctMethod          CatTransMethod  
 0  Equal-Width (Bucket)                          
 1  Equal-Width (Bucket)                          
 2                        Label (Sparse One-Hot)  
 3                        Label (Sparse One-Hot)  
 4                  MDLP                          
 5                                         DTree  

[VarTransInfo]

 Variable Transformation Information for KDD98
 
      Variable  VarIndex Interaction Transformation        Input1  \
 0    RAMNT_10       NaN          no           hshm                 
 1    RAMNT_13       NaN          no           hshm                 
 2    RAMNT_16       NaN          no           hshm                 
 3    RAMNT_17       NaN          no           hshm                 
 4    RAMNT_18       NaN          no           hshm                 
 ..        ...       ...         ...            ...           ...   
 266                60.0         yes         inter1  tr1_CONTROLN   
 267                61.0         yes         inter1  tr1_CONTROLN   
 268                62.0         yes         inter1  tr1_CONTROLN   
 269                63.0         yes         inter1  tr1_CONTROLN   
 270                64.0         yes         inter1  tr1_CONTROLN   
 
            Input2        EvalVar      ResultVar      N  NMiss  \
 0                        TARGETB  hshm_RAMNT_10  10461  84951   
 1                        TARGETB  hshm_RAMNT_13  12250  83162   
 2                        TARGETB  hshm_RAMNT_16  26994  68418   
 3                        TARGETB  hshm_RAMNT_17   9401  86011   
 4                        TARGETB  hshm_RAMNT_18  19778  75634   
 ..            ...            ...            ...    ...    ...   
 266       tr1_IC2  tar_p_TARGETB  inter1_var_60  95412      0   
 267       tr1_IC3  tar_p_TARGETB  inter1_var_61  95412      0   
 268       tr1_IC4  tar_p_TARGETB  inter1_var_62  95412      0   
 269       tr1_IC5  tar_p_TARGETB  inter1_var_63  95412      0   
 270  tr1_CONTROLN  tar_p_TARGETB  inter1_var_64  95412      0   
 
     ImputedValueNominal  NBins  
 0                          6.0  
 1                          6.0  
 2                          6.0  
 3                          6.0  
 4                          6.0  
 ..                  ...    ...  
 266                        5.0  
 267                        5.0  
 268                        5.0  
 269                        5.0  
 270                        5.0  
 
 [271 rows x 12 columns]

[NomVarInfo]

 Nominal Variable Information for KDD98
 
      Variable        N    NMiss  NLevels
 0    ADATE_10  62664.0  32748.0      2.0
 1    ADATE_13  55193.0  40219.0      3.0
 2    ADATE_15  29935.0  65477.0      1.0
 3    ADATE_17  67762.0  27650.0      3.0
 4    ADATE_19  70932.0  24480.0      3.0
 ..        ...      ...      ...      ...
 149      TPE7  95412.0      0.0     18.0
 150  VETERANS  10426.0  84986.0      1.0
 151    WALKER

In [ ]:
# apply the scoring object to future/validation/test etc slices ... 


In [40]:
# in this case we are simply showing how to apply it (on the same table /train tbl - replace with test/valid etc)
s.astore.score(table=input_tbl,
               rstore='scoring_model',
               # put whatever vars you want to copy here
               copyvars = [target],
               casout=dict(name='out1', replace=True))

NOTE: Executing action 'astore.score'.
NOTE: Action 'astore.score' used (Total process time):
NOTE:       real time               0.341808 seconds
NOTE:       cpu time                6.020083 seconds (1761.25%)
NOTE:       total nodes             7 (224 cores)
NOTE:       total memory            1.72T
NOTE:       memory                  403.52M (0.02%)
NOTE:       bytes moved             198.03M


[OutputCasTables]

                 casLib  Name   Rows  Columns  \
 0  CASUSERHDFS(bigebr)  out1  95412      272   
 
                                          casTable  
 0  CASTable('out1', caslib='CASUSERHDFS(bigebr)')  

[Timing]

 Task Timing
 
                  Task   Seconds   Percent
 0   Loading the Store  0.006435  0.019672
 1  Creating the State  0.032220  0.098499
 2             Scoring  0.287183  0.877941
 3               Total  0.327110  1.000000

+ Elapsed: 0.342s, user: 4.98s, sys: 1.04s, mem: 404mb

In [41]:
s.close()